In [1]:
import os
import time
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.autograd import Variable

### System properties and libs currently in use
- We have developed using python 3.5.x, pytorch 0.2.1
- No significant attention was given to backwards compatibility

In [2]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
print('Active CUDA Device: GPU', torch.cuda.current_device())

__Python VERSION: 3.5.3 |Anaconda 4.4.0 (64-bit)| (default, Mar  6 2017, 11:58:13) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
__pyTorch VERSION: 0.2.0_4
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0


### Utilities
- Saving images and models

In [3]:
def save_images(netG, fixed_noise, outputDir,epoch):
    '''
    Generates a batch of images from the given 'noise'.
    Saves 64 of the generated samples to 'outputDir' system path.
    Inputs are the network (netG), a 'noise' input, system path to which images will be saved (outputDir) and current 'epoch'.
    '''
    noise = Variable(fixed_noise)
    netG.eval()
    fake = netG(noise)
    netG.train()
    vutils.save_image(fake.data[0:64,:,:,:],'%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)

def save_models(netG, netD, outputDir, epoch):
    '''
    Saves model state dictionary for generator and discriminator networks.
    Inputs are the networks (netG, netD), the system path in which to save(outputDir) and the current 'epoch'.
    '''
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [4]:
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


### Output Directory
This is where images will be saved to.

If directory does not exist, it is created.

In [5]:
outputDir = 'outputdir_train_classifier_lotufo'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

OS error: [Errno 17] File exists: 'outputdir_train_classifier_lotufo'


### Dataset definition and hyperparameter setting
- Changing dataset name alters network architecture parameters
- Currently supporting few datasets
- Hyperparameters defined according to Radford et al. (2015)

In [6]:
batch_size = 128

chosen_dataset = 'MNIST'

datasets = {
    'MNIST': torchvision.datasets.MNIST,
    'CIFAR10': torchvision.datasets.CIFAR10,
    'ANIME': '/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
}

dataset = datasets[chosen_dataset]

In [7]:
possible_parameters = {
    'MNIST': {
        'ndf': 64,
        'ngf': 64,
        'nz': 100,
        'nc': 1,
        'imageSize': 64,
        'n_classes' : 10,
        'ngpu': 1,
    },
    'CIFAR10': {
        'ndf': 64,
        'ngf': 64,
        'nz': 100,
        'nc': 3,
        'imageSize' : 64,
        'n_classes' : 10,
        'ngpu' : 1,
    },
    'ANIME': {
        'nc' : 3,
        'ngpu' : 1,
        'nz' : 100,
        'ngf' : 64,
        'ndf' : 64,
        'imageSize' : 64,
        'n_classes' : 1
    }
}

In [8]:
ngf = possible_parameters[chosen_dataset]['ngf']
ndf = possible_parameters[chosen_dataset]['ndf']
nz = possible_parameters[chosen_dataset]['nz']
nc = possible_parameters[chosen_dataset]['nc']
imageSize = possible_parameters[chosen_dataset]['imageSize']
n_classes = possible_parameters[chosen_dataset]['n_classes']
ngpu = possible_parameters[chosen_dataset]['ngpu']

## Creating the Dataset!

In [9]:
if dataset == 'ANIME':
    dataset = torchvision.datasets.ImageFolder(
        root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
        transform=transforms.Compose([
                transforms.Scale((imageSize, imageSize)),
                transforms.ToTensor(),
            ])
    )
else:
    transform = transforms.Compose([
                    transforms.Scale((imageSize, imageSize)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
                ]) 
    dataset_done = dataset('./datasets', train=True, download=True, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset_done, batch_size=batch_size, shuffle=True, num_workers=4)
print('Dataloader length:', len(dataloader))
print("Dataset:", dataloader.dataset)


Dataloader length: 469
Dataset: <torchvision.datasets.mnist.MNIST object at 0x7f89137322e8>


## Definição dos modelos
- Model is a DCGAN
- Images are sized (nc, 64, 64)

In [10]:
class _netD_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf, n_classes):
        super(_netD_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes+1,kernel_size=4,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, inplace=True)
        x = F.leaky_relu(self.batch2(self.conv2(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch3(self.conv3(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch4(self.conv4(x)), 0.2, inplace=True)
        
        x = self.final_conv(x)
        return(x)

In [11]:
class _netG_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf):
        super(_netG_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.convt1 = nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=4, stride=1, padding=0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf*8)
        self.convt2 = nn.ConvTranspose2d(in_channels=ngf * 8, out_channels=ngf * 4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf*4)
        self.convt3 = nn.ConvTranspose2d(in_channels=ngf * 4, out_channels=ngf * 2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf*2)
        self.convt4 = nn.ConvTranspose2d(in_channels=ngf*2, out_channels=ngf, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ngf)
        
        self.final_convt = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc, kernel_size=4, stride=2, padding=1, bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.batch1(self.convt1(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch2(self.convt2(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch3(self.convt3(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch4(self.convt4(x)), 0.2, inplace=True)
        
        x = self.final_convt(x)
        x = F.tanh(x)
        return (x)

In [12]:
netG = _netG_DCGAN(ngpu, nz, nc, ngf)
netD = _netD_DCGAN(ngpu, nz, nc, ndf, n_classes)

## Inicializador de pesos

In [13]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [14]:
netG.apply(weights_init)
netD.apply(weights_init)
print(netG, '\n', netD)

_netG_DCGAN (
  (convt1): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (batch1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (convt2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (convt3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (convt4): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (final_convt): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
) 
 _netD_DCGAN (
  (conv1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch2): BatchNorm2d(128, eps=

## Losses
- Binary Cross-Entropy is used to differentiate real and fake images
- Class loss should be Cross-Entropy

In [15]:
criterion = nn.CrossEntropyLoss()

## Sizes of the tensors

In [16]:
input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print('Input images size:', input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
print('Code size:', noise.size())

Input images size: torch.Size([128, 3, 64, 64])
Code size: torch.Size([128, 100, 1, 1])


In [17]:
label = torch.LongTensor(batch_size,n_classes)
print('Label size:', label.size())
fake_label = 10
real_label = 1

Label size: torch.Size([128, 10])


## Broadcast to GPU

In [18]:
if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion = criterion.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

## Optimizer Parameters
- Following the lead of Radford et al., 2015:

    <b>
    1. beta1 = 0.5
    2. beta2 = 0.999
    3. lr = 0.0002
    </b>

In [19]:
beta1, beta2 = 0.5, 0.999
lr = 2.0e-4

optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))


## Treinamento 

In [20]:
def train_gan(num_epochs, dataloader, netD, netG, outputDir,
              real_labelSmooth=0, epoch_interval=100, D_steps=1, G_steps=1, decay = 0.05):
    
    # This validation is subjective. WGAN-GP uses 100 steps on the critic (netD).
    assert D_steps < 5, "Keep it low, D_steps is too high."
    assert G_steps < 3, "Keep it low, G_steps is too high."
    #assert batch_size % D_steps == 0, "Use batch_size multiple of D_steps."
    real_label = 1
    print('Lets train!')
    for epoch in range(num_epochs):
        start_iter = time.time()  
        D_x = 0
        D_G_z1 = 0
        D_G_z2 = 0
        errD_acum = 0
        errG_acum = 0
        
        print('In epoch = ', epoch, 'real_label_smooth = ',real_labelSmooth )
        for batch, data in enumerate(dataloader, 0):
            if (epoch == 0 and batch == 0):
                    vutils.save_image(data[0][0:64,:,:,:], '%s/real_samples.png' % outputDir, nrow=8)
            if epoch < 25:   
                real_labelSmooth = np.maximum(real_labelSmooth * (1 - 0.05*epoch), 0)
            else:
                real_labelSmooth = 0
            for step in range(D_steps):
                #############################################################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                # 1A - Train the detective network in the Real Dataset
                #############################################################
                netD.zero_grad()
                start = step*(int(data[0].size()[0]/D_steps))
                end = (step+1)*int(data[0].size()[0]/D_steps)
                
                real_cpu = data[0][start:end]
                real_cpu = real_cpu.cuda()
                batch_size = real_cpu.size(0)
                if np.random.random_sample() > real_labelSmooth:
                    target = data[1][start:end].long().cuda()
                else:
                     target = torch.from_numpy(np.random.randint(0, n_classes, batch_size)).type(torch.LongTensor).cuda()
                
                input, label = Variable(real_cpu), Variable(target)

                output = netD(input)
                errD_real = criterion(output.squeeze(),label)
                errD_real.backward()
                
                D_x += output.data.mean()
                
                #######################################################
                # 1B - Train the detective network in the False Dataset
                #######################################################
                
                noise = Variable(torch.FloatTensor(batch_size, nz, 1, 1).normal_(0,1).cuda())
                fake = netG(noise)
                label = Variable(torch.ones(batch_size).long().fill_(fake_label).cuda())
                output = netD(fake.detach()) # ".detach()" to avoid backprop through G
                errD_fake = criterion(output.squeeze(), label)
                errD_fake.backward() # gradients for fake and real data will be accumulated
                
                D_G_z1 += output.data.mean()
                errD_acum += errD_real.data[0] + errD_fake.data[0]
                optimizerD.step()

            for step in range(G_steps):
                ####################################################################################
                # (2) Update G network: maximize log(D(G(z)))
                # Train the faker with the output from the Detective (but don't train the Detective)
                ####################################################################################
                
                netG.zero_grad()
                label = Variable(torch.from_numpy(np.random.randint(0, n_classes, batch_size)).type(torch.LongTensor).cuda())
                output = netD(fake)
                errG = criterion(output.squeeze(), label)
                errG.backward()
                
                D_G_z2 += output.data.mean()
                errG_acum += errG.data[0]
                optimizerG.step()

        print('epoch = ',epoch)

        end_iter = time.time()        

        print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
              % (epoch, num_epochs, errD_acum/D_steps, errG_acum/G_steps, D_x, D_G_z1, D_G_z2, end_iter-start_iter))

        #Save a grid with the pictures from the dataset, up until 64
        save_images(netG = netG, fixed_noise=  fixed_noise, outputDir = outputDir, epoch = epoch)

        if epoch % epoch_interval == 0:
            # do checkpointing
            save_models(netG = netG, netD = netD, outputDir = outputDir, epoch = epoch)
            

In [21]:
outputDir = 'outputdir_train_classifier_0d25'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

In [22]:
num_epochs = 25
real_labelSmooth = 0.25
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.25


Process Process-3:
Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-4:
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(

KeyboardInterrupt: 

In [ ]:
outputDir = 'outputdir_train_classifier_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.15

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

In [ ]:
outputDir = 'outputdir_train_classifier_0d20'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.20

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

In [49]:
outputDir = 'outputdir_train_classifier_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.10

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

Lets train!
In epoch =  0 real_label_smooth =  0.1
epoch =  0
[0/100] Loss_D: 489.9057 Loss_G: 6355.2549 D(x): 1892.3792 D(G(z)): 1325.7127 / 1266.0529 Elapsed 43.89 s
In epoch =  1 real_label_smooth =  0.1
epoch =  1
[1/100] Loss_D: 24.7583 Loss_G: 4765.3966 D(x): 1001.4163 D(G(z)): 762.1362 / 735.1085 Elapsed 44.00 s
In epoch =  2 real_label_smooth =  3.56755666949e-12
epoch =  2
[2/100] Loss_D: 2.0290 Loss_G: 5438.1060 D(x): 74.0035 D(G(z)): 246.1375 / 228.5548 Elapsed 43.79 s
In epoch =  3 real_label_smooth =  1.23625427814e-33
epoch =  3
[3/100] Loss_D: 0.9688 Loss_G: 5460.8529 D(x): -367.6159 D(G(z)): -327.2970 / -339.1286 Elapsed 44.12 s
In epoch =  4 real_label_smooth =  9.76301524616e-67
epoch =  4
[4/100] Loss_D: 0.5046 Loss_G: 6229.2302 D(x): -588.4061 D(G(z)): -683.5881 / -691.7062 Elapsed 43.93 s
In epoch =  5 real_label_smooth =  3.45770443308e-112
epoch =  5
[5/100] Loss_D: 27.2388 Loss_G: 5803.1441 D(x): -238.4102 D(G(z)): -440.2289 / -473.9404 Elapsed 44.07 s
In epoch 

Process Process-667:
Process Process-668:
Process Process-665:
Process Process-666:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/gabriel/anaconda3/envs/py35/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self.

KeyboardInterrupt: 

KeyboardInterrupt


In [53]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d1'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.1
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

OS error: [Errno 17] File exists: 'outputdir_train_classifier_decay_0d025_label_0d1'
Lets train!
In epoch =  0 real_label_smooth =  0.1
epoch =  0
[0/40] Loss_D: 299.6264 Loss_G: 3986.2494 D(x): 4684.5141 D(G(z)): 4462.2574 / 4436.1643 Elapsed 44.07 s
In epoch =  1 real_label_smooth =  0.1
epoch =  1
[1/40] Loss_D: 17.6498 Loss_G: 4523.2620 D(x): 4342.3166 D(G(z)): 4626.4624 / 4612.3886 Elapsed 44.15 s
In epoch =  2 real_label_smooth =  3.56755666949e-12
epoch =  2
[2/40] Loss_D: 1.6169 Loss_G: 5395.4594 D(x): 3517.9936 D(G(z)): 3330.3491 / 3322.5670 Elapsed 43.94 s
In epoch =  3 real_label_smooth =  1.23625427814e-33
epoch =  3
[3/40] Loss_D: 0.8399 Loss_G: 6535.2582 D(x): 2754.9849 D(G(z)): 1918.5237 / 1910.1450 Elapsed 43.93 s
In epoch =  4 real_label_smooth =  9.76301524616e-67
epoch =  4
[4/40] Loss_D: 0.5566 Loss_G: 6048.7730 D(x): 2248.0342 D(G(z)): 2086.4295 / 2079.0269 Elapsed 43.98 s
In epoch =  5 real_label_smooth =  3.45770443308e-112
epoch =  5
[5/40] Loss_D: 0.2449 Loss_G

In [54]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.15
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.15
epoch =  0
[0/40] Loss_D: 508.5954 Loss_G: 4500.6372 D(x): 5881.1898 D(G(z)): 5872.5133 / 5836.2248 Elapsed 44.28 s
In epoch =  1 real_label_smooth =  0.15
epoch =  1
[1/40] Loss_D: 13.0091 Loss_G: 4565.4925 D(x): 5494.6438 D(G(z)): 5756.3354 / 5742.2552 Elapsed 43.97 s
In epoch =  2 real_label_smooth =  5.35133500423e-12
epoch =  2
[2/40] Loss_D: 2.1132 Loss_G: 5154.6022 D(x): 4736.9500 D(G(z)): 4080.6326 / 4071.0461 Elapsed 43.98 s
In epoch =  3 real_label_smooth =  1.85438141721e-33
epoch =  3
[3/40] Loss_D: 0.9438 Loss_G: 5502.7155 D(x): 4038.5562 D(G(z)): 3246.6930 / 3238.7257 Elapsed 44.10 s
In epoch =  4 real_label_smooth =  1.46445228692e-66
epoch =  4
[4/40] Loss_D: 15.7246 Loss_G: 5726.9193 D(x): 3399.8026 D(G(z)): 3418.6800 / 3394.4242 Elapsed 43.93 s
In epoch =  5 real_label_smooth =  5.18655664962e-112
epoch =  5
[5/40] Loss_D: 1.5668 Loss_G: 6275.4745 D(x): 2308.9835 D(G(z)): 1944.6503 / 1933.4710 Elapsed 44.10 s
In epoc

In [55]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d20'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.20
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.2
epoch =  0
[0/40] Loss_D: 714.9876 Loss_G: 3808.0237 D(x): 7127.4171 D(G(z)): 6878.5448 / 6874.1314 Elapsed 44.11 s
In epoch =  1 real_label_smooth =  0.2
epoch =  1
[1/40] Loss_D: 55.3161 Loss_G: 4500.3197 D(x): 6800.7863 D(G(z)): 6685.2322 / 6675.9286 Elapsed 53.31 s
In epoch =  2 real_label_smooth =  7.13511333898e-12
epoch =  2
[2/40] Loss_D: 2.0881 Loss_G: 5857.2087 D(x): 5035.7761 D(G(z)): 5607.3286 / 5601.8534 Elapsed 80.18 s
In epoch =  3 real_label_smooth =  2.47250855628e-33
epoch =  3
[3/40] Loss_D: 0.8350 Loss_G: 6580.7040 D(x): 3722.6656 D(G(z)): 4425.2984 / 4423.1513 Elapsed 44.00 s
In epoch =  4 real_label_smooth =  1.95260304923e-66
epoch =  4
[4/40] Loss_D: 0.7148 Loss_G: 7119.6863 D(x): 3047.9679 D(G(z)): 3187.5652 / 3184.8675 Elapsed 52.48 s
In epoch =  5 real_label_smooth =  6.91540886616e-112
epoch =  5
[5/40] Loss_D: 0.4383 Loss_G: 7197.2645 D(x): 2441.4306 D(G(z)): 2539.3882 / 2536.0665 Elapsed 81.66 s
In epoch =

In [56]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d25'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.25
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.25
epoch =  0
[0/40] Loss_D: 819.8724 Loss_G: 3617.0136 D(x): 7135.3925 D(G(z)): 7245.4033 / 7240.3889 Elapsed 44.21 s
In epoch =  1 real_label_smooth =  0.25
epoch =  1
[1/40] Loss_D: 44.5112 Loss_G: 4426.5321 D(x): 6857.5648 D(G(z)): 6502.6205 / 6493.4633 Elapsed 44.05 s
In epoch =  2 real_label_smooth =  8.91889167372e-12
epoch =  2
[2/40] Loss_D: 2.3197 Loss_G: 6005.3184 D(x): 5123.9166 D(G(z)): 5028.4857 / 5022.7162 Elapsed 43.92 s
In epoch =  3 real_label_smooth =  3.09063569535e-33
epoch =  3
[3/40] Loss_D: 0.5356 Loss_G: 7082.0629 D(x): 3333.4222 D(G(z)): 3285.5788 / 3282.3544 Elapsed 44.05 s
In epoch =  4 real_label_smooth =  2.44075381154e-66
epoch =  4
[4/40] Loss_D: 0.3983 Loss_G: 6864.5100 D(x): 2101.6130 D(G(z)): 3885.7073 / 3882.2587 Elapsed 44.04 s
In epoch =  5 real_label_smooth =  8.6442610827e-112
epoch =  5
[5/40] Loss_D: 0.0863 Loss_G: 7002.1890 D(x): 2075.4041 D(G(z)): 4631.5733 / 4630.1305 Elapsed 44.05 s
In epoch 

In [57]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d30'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.30
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.3
epoch =  0
[0/40] Loss_D: 781.7199 Loss_G: 3715.1845 D(x): 10748.5893 D(G(z)): 11206.2752 / 11218.2634 Elapsed 44.02 s
In epoch =  1 real_label_smooth =  0.3
epoch =  1
[1/40] Loss_D: 49.6429 Loss_G: 4539.6461 D(x): 10675.3030 D(G(z)): 10760.4706 / 10755.2856 Elapsed 43.93 s
In epoch =  2 real_label_smooth =  1.07026700085e-11
epoch =  2
[2/40] Loss_D: 2.7163 Loss_G: 5265.9083 D(x): 9479.4939 D(G(z)): 9948.6701 / 9944.2290 Elapsed 44.01 s
In epoch =  3 real_label_smooth =  3.70876283442e-33
epoch =  3
[3/40] Loss_D: 0.8992 Loss_G: 6183.0167 D(x): 8002.1903 D(G(z)): 8749.0909 / 8744.7351 Elapsed 43.98 s
In epoch =  4 real_label_smooth =  2.92890457385e-66
epoch =  4
[4/40] Loss_D: 0.6318 Loss_G: 7038.9538 D(x): 6699.0375 D(G(z)): 7950.7834 / 7947.7233 Elapsed 43.94 s
In epoch =  5 real_label_smooth =  1.03731132992e-111
epoch =  5
[5/40] Loss_D: 32.3553 Loss_G: 6709.2191 D(x): 5080.7122 D(G(z)): 5468.9367 / 5454.8845 Elapsed 43.84 s
In 

In [58]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.15
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

OS error: [Errno 17] File exists: 'outputdir_train_classifier_decay_0d025_label_0d15'
Lets train!
In epoch =  0 real_label_smooth =  0.15
epoch =  0
[0/40] Loss_D: 549.7038 Loss_G: 3797.2124 D(x): 12137.1745 D(G(z)): 12496.9786 / 12504.5495 Elapsed 43.18 s
In epoch =  1 real_label_smooth =  0.15
epoch =  1
[1/40] Loss_D: 17.5712 Loss_G: 4774.6924 D(x): 11105.6962 D(G(z)): 11200.3752 / 11193.0390 Elapsed 44.03 s
In epoch =  2 real_label_smooth =  5.35133500423e-12
epoch =  2
[2/40] Loss_D: 1.6773 Loss_G: 6349.0323 D(x): 8425.8559 D(G(z)): 9316.0792 / 9309.6890 Elapsed 43.85 s
In epoch =  3 real_label_smooth =  1.85438141721e-33
epoch =  3
[3/40] Loss_D: 0.6435 Loss_G: 8157.9107 D(x): 6153.2338 D(G(z)): 7238.1553 / 7235.1724 Elapsed 44.05 s
In epoch =  4 real_label_smooth =  1.46445228692e-66
epoch =  4
[4/40] Loss_D: 0.3276 Loss_G: 8864.2031 D(x): 5123.3387 D(G(z)): 6122.1273 / 6120.8418 Elapsed 44.41 s
In epoch =  5 real_label_smooth =  5.18655664962e-112
epoch =  5
[5/40] Loss_D: 0.44

In [59]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.1
epoch =  0
[0/40] Loss_D: 507.7420 Loss_G: 3972.4226 D(x): 12859.3070 D(G(z)): 13087.5091 / 13103.0339 Elapsed 45.55 s
In epoch =  1 real_label_smooth =  0.1
epoch =  1
[1/40] Loss_D: 29.4912 Loss_G: 4546.8625 D(x): 12531.2498 D(G(z)): 12373.2577 / 12368.0497 Elapsed 45.32 s
In epoch =  2 real_label_smooth =  3.56755666949e-12
epoch =  2
[2/40] Loss_D: 2.5138 Loss_G: 5855.9885 D(x): 11256.2571 D(G(z)): 10987.5664 / 10984.3815 Elapsed 45.64 s
In epoch =  3 real_label_smooth =  1.23625427814e-33
epoch =  3
[3/40] Loss_D: 1.2730 Loss_G: 6718.6383 D(x): 9944.6017 D(G(z)): 9359.6083 / 9356.5230 Elapsed 45.50 s
In epoch =  4 real_label_smooth =  9.76301524616e-67
epoch =  4
[4/40] Loss_D: 0.4384 Loss_G: 7332.0316 D(x): 8878.8635 D(G(z)): 8392.9286 / 8390.6383 Elapsed 45.48 s
In epoch =  5 real_label_smooth =  3.45770443308e-112
epoch =  5
[5/40] Loss_D: 0.2238 Loss_G: 7362.7106 D(x): 7910.5741 D(G(z)): 8187.9386 / 8185.6142 Elapsed 45.41 s
I

In [ ]:
outputDir = 'outputdir_train_classifier_decay_0d018_label_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.1
epoch =  0
[0/40] Loss_D: 375.8008 Loss_G: 4548.2325 D(x): 12897.6643 D(G(z)): 13016.5312 / 13031.5164 Elapsed 43.58 s
In epoch =  1 real_label_smooth =  0.1
epoch =  1
[1/40] Loss_D: 17.1547 Loss_G: 4886.9106 D(x): 12959.3268 D(G(z)): 12558.4153 / 12555.0399 Elapsed 43.77 s
In epoch =  2 real_label_smooth =  3.56755666949e-12
epoch =  2
[2/40] Loss_D: 1.9574 Loss_G: 5766.4303 D(x): 12290.9605 D(G(z)): 12137.2978 / 12135.5654 Elapsed 43.93 s
In epoch =  3 real_label_smooth =  1.23625427814e-33
epoch =  3
[3/40] Loss_D: 1.0432 Loss_G: 6312.5672 D(x): 11584.6274 D(G(z)): 11173.1914 / 11171.2680 Elapsed 44.29 s
In epoch =  4 real_label_smooth =  9.76301524616e-67
epoch =  4
[4/40] Loss_D: 0.7644 Loss_G: 6926.1553 D(x): 10837.1461 D(G(z)): 10399.9942 / 10398.4188 Elapsed 44.67 s
In epoch =  5 real_label_smooth =  3.45770443308e-112
epoch =  5
[5/40] Loss_D: 0.4421 Loss_G: 7380.4011 D(x): 10066.3445 D(G(z)): 9456.5590 / 9454.4426 Elapsed 44

In [ ]:
outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

OS error: [Errno 17] File exists: 'outputdir_train_classifier_decay_0d025_label_0d15'
Lets train!
In epoch =  0 real_label_smooth =  0.1
epoch =  0
[0/40] Loss_D: 586.5647 Loss_G: 3512.8855 D(x): -310.1243 D(G(z)): -371.3154 / -378.8754 Elapsed 43.00 s
In epoch =  1 real_label_smooth =  0.1
epoch =  1
[1/40] Loss_D: 196.4603 Loss_G: 3993.8315 D(x): -532.8350 D(G(z)): -568.7550 / -571.3881 Elapsed 43.42 s
In epoch =  2 real_label_smooth =  3.56755666949e-12
epoch =  2
[2/40] Loss_D: 122.9132 Loss_G: 4885.0473 D(x): -727.9536 D(G(z)): -771.0179 / -773.6365 Elapsed 43.39 s
In epoch =  3 real_label_smooth =  1.23625427814e-33
epoch =  3
[3/40] Loss_D: 136.9712 Loss_G: 5337.2350 D(x): -916.5261 D(G(z)): -954.3983 / -956.4158 Elapsed 43.60 s
In epoch =  4 real_label_smooth =  9.76301524616e-67
epoch =  4
[4/40] Loss_D: 110.3570 Loss_G: 5728.3650 D(x): -1082.2183 D(G(z)): -1130.1459 / -1134.7157 Elapsed 43.90 s
In epoch =  5 real_label_smooth =  3.45770443308e-112
epoch =  5
[5/40] Loss_D: 12